# Import Agg Performance Nationwide Data

In [1]:
import cx_Oracle
import pandas as pd
import numpy as np

In [2]:
conStr = 'NARUT_E01044451/v9VzR8xsuwp#tyTb@TDM-scan.dc2.true.th:1521/TDMDBPR'
conn = cx_Oracle.connect(conStr)
print('TDMDBPR : Connected')
cur = conn.cursor()

v_week = (2023021, )


try:
    # print("Database version:", conn.version)
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, COMP_CD, METRIC_CD, METRIC_NAME, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACH_SNAP, GAP_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, ACH_AGG, GAP_AGG, WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, YOY, YOY_PERCENT, RR, RR_ACH, WTD, MTD, QTD, YTD, CENTER, METRIC_GRP, PRODUCT_GRP, SEQ, PRIORITY, REMARK, AREA_TYPE, AREA_CD, AREA_NAME, REMARK_ACTUAL, REMARK_TARGET, PPN_TM, METRIC_NAME_GROUP
        FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK
        WHERE AREA_TYPE = 'P'
    """
    
    cur.execute(sqlTxt)

    print(f'\nDone !!!')

    rows = cur.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print('\nTDMDBPR : Disconnected')

TDMDBPR : Connected

Execute query as Week 2023021...

Done !!!

TDMDBPR : Disconnected


In [3]:
df.shape

(46080, 54)

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [5]:
df.head()

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,DAY_DESC,MONTH_DESC,QUARTER_DESC,COMP_CD,METRIC_CD,...,SEQ,PRIORITY,REMARK,AREA_TYPE,AREA_CD,AREA_NAME,REMARK_ACTUAL,REMARK_TARGET,PPN_TM,METRIC_NAME_GROUP
0,20230301,2023009,202303,20231,2023,Wednesday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-30 08:50:36,% After-Sales Service via Digital Channels
1,20230302,2023009,202303,20231,2023,Thursday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-30 08:50:36,% After-Sales Service via Digital Channels
2,20230303,2023009,202303,20231,2023,Friday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-30 08:50:36,% After-Sales Service via Digital Channels
3,20230304,2023009,202303,20231,2023,Saturday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-30 08:50:36,% After-Sales Service via Digital Channels
4,20230305,2023009,202303,20231,2023,Sunday,March,Quarter 1,TRUE,DIS10015,...,7015.0,High,None,P,P,Nationwide,None,None,2023-05-30 08:50:36,% After-Sales Service via Digital Channels


## Generate Output file

In [6]:
# C:\Users\Narut4\coding\Jupyter\Work with Oracle.ipynb

output_name = f'Raw-Agg Perf Nationwide_Wk {v_week[0]}'

df.to_csv(f'{output_name}.csv', index=False, encoding='utf-8')

print(f'\n"{output_name}.csv" is generated')


"Raw-Agg Perf Nationwide_Wk 2023021.csv" is generated
